# Random Forest

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import pydot

Открываем данные
База данных содержит 76 атрибутов. Переменная «target» относится к наличию болезни сердца у пациента. Целочисленное значение от 0 (нет сердечных заболеваний) до 1.

In [2]:
data = pd.read_csv('heart.csv')

In [3]:
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
# age - in years
# sex - (1 = male; 0 = female)
# cp - chest pain type
# trestbps - resting blood pressure (in mm Hg on admission to the hospital)
# chol - serum cholestoral in mg/dl
# fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
# restecg - resting electrocardiographic results
# thalach - maximum heart rate achieved
# exang - exercise induced angina (1 = yes; 0 = no)
# oldpeak - ST depression induced by exercise relative to rest
# slope - the slope of the peak exercise ST segment
# ca - number of major vessels (0-3) colored by flourosopy
# thal - 3 = normal; 6 = fixed defect; 7 = reversable defect
# target - presence of heart disease (1 or 0)

In [5]:
data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [6]:
data.target.value_counts()

1    165
0    138
Name: target, dtype: int64

Подготока данных

In [7]:
# Target - зависимая переменная (бинарный классификатор)
target = np.array(data['target'])

# Исключаем зависимую переменную из матрицы переменных
data = data.drop('target', axis = 1)

# Сохраняем список с названиями переменных для дальнейшего использования
data_list = list(data.columns)

# Конвертируем в массив numpy
data = np.array(data)

In [8]:
# Разделяем на обучающую и тестовую выборки

train_features, test_features, train_target, test_target = train_test_split(data, 
                                                                            target, 
                                                                            test_size = 0.2, 
                                                                            random_state = 42)

Кросс-валидация для определения лучших параметров модели

In [9]:
# Определение лучших параметров модели

params = {'max_depth': [3,5,7,10], 
          'n_estimators': [10, 100, 500, 1000, 1500]}
rf = RandomForestClassifier(random_state = 42, n_jobs=-1)
grid_search = GridSearchCV(estimator = rf, param_grid = params, cv = 3, n_jobs = -1)
grid_search.fit(train_features, train_target)

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=-1,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_

In [10]:
grid_search.best_params_

{'max_depth': 3, 'n_estimators': 100}

Обучение модели Random Forest

In [11]:
# Создаем модель с 100 деревьев с максимальной глубиной 3
rf = RandomForestClassifier(n_estimators = 100, max_depth = 3, random_state = 42)

In [12]:
# Обучаем модель на обучающей выборке
rf.fit(train_features, train_target);

Проверка качества модели на тестовой выборке

In [13]:
from sklearn.metrics import accuracy_score

In [14]:
predictions = rf.predict(test_features)
accuracy = accuracy_score(test_target, predictions)
print(accuracy)

0.8852459016393442


Визуализация дерева

In [ ]:
from sklearn.tree import export_graphviz
tree = rf.estimators_[5]
export_graphviz(tree, out_file = 'tree.dot', feature_names = data_list, rounded = True, precision = 1)